# Graph generation

In [1]:
#import warnings
#warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

from utils import textual_analysis

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import csv
import datetime
from time import time

In [2]:
class Struttura(object):
    """
    puo' essere una pa o un agg (aggiudicatario)
    """
    def __init__(self, id_s=None, codiceFiscale=None, tipoStruttura=None, outlinks=None, oggetto=None, tfidfAbstract=None):
        self.id_s = id_s
        self.codiceFiscale = codiceFiscale
        self.tipoStruttura = tipoStruttura
        self.outlinks = outlinks
        self.oggetto = oggetto
        self.tfidfAbstract = tfidfAbstract
        
    def __str__(self):
        return "id: " + str(self.id_s) + ", cf: " +  str(self.codiceFiscale) + ", tipo: " + str(self.tipoStruttura) + ", outlinks: "  + str(self.outlinks) + ", terms: " + str(self.tfidfAbstract)       

class Edge(object):
    def __init__(self, src=None, dst=None, score=None):
        self.src = src
        self.dst = dst
        self.score = score
    def __str__(self):
        return str(self.src) + " -> " +  str(self.dst) + " : " + str(self.score)   

def from_oggettiAggregati_to_Struttura(id_series, knn, bestTermsStr, seriesOggettiAggregati):
    return Struttura(id_s=id_series, 
                     codiceFiscale = seriesOggettiAggregati["codiceFiscaleStruttura"].strip(),
                     tipoStruttura = seriesOggettiAggregati["tipoStruttura"], 
                     outlinks = knn, 
                     oggetto = seriesOggettiAggregati["oggetto"], 
                     tfidfAbstract = bestTermsStr)
  

In [3]:
def getMinOrDefault(array, default_value, threshold = 0.6):
    """
    return the index of the smallest element compared to the threshold
    """
    res = D.shape[1]
    try:
        res = np.min(np.where(array < threshold))
    except:
        return res
    return res

In [4]:
df_file_name = "data/oggetti_aggregati.pickle"

t = time()
df = pd.read_pickle(df_file_name)
total = time() - t

print("Dataset loaded in %0.3f sec" % total)

df.info()

Dataset loaded in 0.327 sec
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307265 entries, 0 to 307264
Data columns (total 3 columns):
codiceFiscaleStruttura    307265 non-null object
oggetto                   307265 non-null object
tipoStruttura             307265 non-null object
dtypes: object(3)
memory usage: 7.0+ MB


In [5]:
import pickle
tf = pickle.load(open("data/n6/tf.pickle", "rb"))

import scipy
m = scipy.sparse.load_npz('data/n6/sparse_matrix_tfidf.npz')

In [6]:
I = np.load("data/n6/normalized_matrix_embedding_52004terms_weigthedTrue.npysimilarity_I_1000_k.npy")
D = np.load("data/n6/normalized_matrix_embedding_52004terms_weigthedTrue.npysimilarity_D_1000_k.npy")

In [7]:
#filtra KNN con soglia maggiore ad una data threshold
t0 = time()

d = D.shape[1]
min_knn_indices = (getMinOrDefault(row, d) for row in D)
total = time() - t0

print("Min indices done in %0.3f sec" % total)

Min indices done in 0.000 sec


In [8]:
s = ""

for i in df.index:
    tfidf_text = ' '.join(textual_analysis.top_feats_in_doc(m,  tf.get_feature_names(), i ,20))
    data = df.iloc[i]
    knn = next(min_knn_indices)
    s = from_oggettiAggregati_to_Struttura(i, knn, tfidf_text, data)
    edges = (Edge(i, I[i][o], round(D[i][o],2)) for o in range(knn))
    if(i == 10):
        break


In [9]:
print(next(edges))

10 -> 10 : 1.0


In [10]:
print(s)

id: 10, cf: 00597318000100, tipo: AGG, outlinks: 1000, terms: fotografica:0.56 catering:0.49 fiera:0.45 copertura:0.41 servizi:0.27
